In [1]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle
import math
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time
import re
import datetime 
import dateutil.parser
import string

In [2]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [3]:
bayHome_filename = "bay_rig_home_model.pkl"
bayRoad_filename = "bay_rig_road_model.pkl"
ardHome_filename = "ard_home_model.pkl"
ardRoad_filename = "ard_road_model.pkl"
huberHome_filename = "huber_home_model.pkl"
huberRoad_filename = "huber_road_model.pkl"
sgdHome_filename = "sgd_home_model.pkl"
sgdRoad_filename = "sgd_road_model.pkl"
theilHome_filename = "theil_sen_home_model.pkl"
theilRoad_filename = "theil_sen_road_model.pkl"
ransacHome_filename = "ransac_home_model.pkl"
ransacRoad_filename = "ransac_road_model.pkl"

bayHome_model = pickle.load(open(bayHome_filename, 'rb'))
bayRoad_model = pickle.load(open(bayRoad_filename, 'rb'))
ardHome_model = pickle.load(open(ardHome_filename, 'rb'))
ardRoad_model = pickle.load(open(ardRoad_filename, 'rb'))
huberHome_model = pickle.load(open(huberHome_filename, 'rb'))
huberRoad_model = pickle.load(open(huberRoad_filename, 'rb'))
sgdHome_model = pickle.load(open(sgdHome_filename, 'rb'))
sgdRoad_model = pickle.load(open(sgdRoad_filename, 'rb'))
theilHome_model = pickle.load(open(theilHome_filename, 'rb'))
theilRoad_model = pickle.load(open(theilRoad_filename, 'rb'))
ransacHome_model = pickle.load(open(ransacHome_filename, 'rb'))
ransacRoad_model = pickle.load(open(ransacRoad_filename, 'rb'))

In [4]:
temp = db.score_pred_testing_data.find().sort('GameDate',pymongo.DESCENDING)
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df_test = pd.DataFrame(temp)

In [5]:
df_test.columns

Index(['AST%_h', 'AST%_r', 'AST/TO_h', 'AST/TO_r', 'ASTRatio_h', 'ASTRatio_r',
       'DREB%_h', 'DREB%_r', 'DefRtg_h', 'DefRtg_r', 'ELO_h', 'ELO_r',
       'FTARate_h', 'FTARate_r', 'GameDate', 'HomeTeam', 'NetRtg_h',
       'NetRtg_r', 'OREB%_h', 'OREB%_r', 'OffRtg_h', 'OffRtg_r', 'PACE_h',
       'PACE_r', 'PIE_h', 'PIE_r', 'PTS_h', 'PTS_r', 'REB%_h', 'REB%_r',
       'RoadTeam', 'TOV%_h', 'TOV%_r', 'TS%_h', 'TS%_r', 'eFG%_h', 'eFG%_r'],
      dtype='object')

In [6]:
teams = df_test['HomeTeam'].unique()
gamedates = df_test['GameDate'].unique()

In [7]:
X_test = df_test.loc[:,['DefRtg_h', 'PIE_h', 'FTARate_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
                        'DefRtg_r', 'PIE_r', 'FTARate_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
y_h_test = df_test['PTS_h']
y_r_test = df_test['PTS_r']

In [9]:
from sklearn.externals import joblib
scoreScaler_filename = "score_scaler.save"
X_scaler = joblib.load(scoreScaler_filename)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
models = {'bay': [bayRoad_model, bayHome_model], 'ard': [ardRoad_model, ardHome_model], 'huber': [huberRoad_model, huberHome_model], 'sgd': [sgdRoad_model, sgdHome_model], 'theil': [theilRoad_model, theilHome_model]}
scoreHomePred = {'bay': [], 'ard': [], 'huber': [], 'sgd': [], 'theil': []}
scoreRoadPred = {'bay': [], 'ard': [], 'huber': [], 'sgd': [], 'theil': []}
scoreHomeActual = []
scoreRoadActual = []
for index, row in df_test.iterrows():
    date = row['GameDate']
    homeTeam = row['HomeTeam']
    roadTeam = row['RoadTeam']
    homeGames = df_test.loc[(df_test['HomeTeam'] == homeTeam) & (df_test['GameDate'] < date)]
    roadGames = df_test.loc[(df_test['RoadTeam'] == homeTeam) & (df_test['GameDate'] < date)]
    if((len(homeGames) >= 10) & (len(roadGames) >= 10)):
        #print(f"{date}: RoadTeam: {roadTeam} HomeTeam: {homeTeam}")
        #print(homeGames[0:5]['GameDate'])
        homeMean = homeGames[0:10][['DefRtg_h', 'PIE_h', 'FTARate_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h']].mean()
        roadMean = roadGames[0:10][['DefRtg_r', 'PIE_r', 'FTARate_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']].mean()
        test_series = pd.Series(data = [homeMean['DefRtg_h'], homeMean['PIE_h'], homeMean['FTARate_h'], homeMean['REB%_h'], homeMean['TOV%_h'], homeMean['TS%_h'], homeMean['PACE_h'], homeMean['ELO_h'],
                                        roadMean['DefRtg_r'], roadMean['PIE_r'], roadMean['FTARate_r'], roadMean['REB%_r'], roadMean['TOV%_r'], roadMean['TS%_r'], roadMean['PACE_r'], roadMean['ELO_r']
        ])
        test_data = test_series.values.reshape(1,16)
        test_data = X_scaler.transform(test_data)
        scoreHomeActual.append(row['PTS_h'])
        scoreRoadActual.append(row['PTS_r'])
        for model in models:
            scoreRoadPred[model].append(models[model][0].predict(test_data)[0])
            scoreHomePred[model].append(models[model][1].predict(test_data)[0])

In [11]:
len(scoreHomeActual)

536

In [12]:
len(scoreHomePred['bay'])

536

In [13]:
len(scoreRoadActual)

536

In [14]:
len(scoreRoadPred['bay'])

536

In [15]:
d = {'HomeTrue':scoreHomeActual, 'RoadTrue':scoreRoadActual, 
     'HomeBayPred': scoreHomePred['bay'], 'RoadBayPred': scoreRoadPred['bay'], 
     'HomeArdPred': scoreHomePred['ard'], 'RoadArdPred': scoreRoadPred['ard'], 
     'HomeHuberPred': scoreHomePred['huber'], 'RoadHuberPred': scoreRoadPred['huber'], 
     'HomeSgdPred': scoreHomePred['sgd'], 'RoadSgdPred': scoreRoadPred['sgd'], 
     'HomeTheilPred': scoreHomePred['theil'], 'RoadTheilPred': scoreRoadPred['theil']}
test_scores_df = pd.DataFrame(data=d)

In [16]:
test_scores_df.columns

Index(['HomeTrue', 'RoadTrue', 'HomeBayPred', 'RoadBayPred', 'HomeArdPred',
       'RoadArdPred', 'HomeHuberPred', 'RoadHuberPred', 'HomeSgdPred',
       'RoadSgdPred', 'HomeTheilPred', 'RoadTheilPred'],
      dtype='object')

In [17]:
test_scores_df['DeltaTrue'] = test_scores_df['HomeTrue']-test_scores_df['RoadTrue']
test_scores_df['DeltaBayPred'] = test_scores_df['HomeBayPred']-test_scores_df['RoadBayPred']
test_scores_df['DeltaArdPred'] = test_scores_df['HomeArdPred']-test_scores_df['RoadArdPred']
test_scores_df['DeltaHuberPred'] = test_scores_df['HomeHuberPred']-test_scores_df['RoadHuberPred']
test_scores_df['DeltaSgdPred'] = test_scores_df['HomeSgdPred']-test_scores_df['RoadSgdPred']
test_scores_df['DeltaTheilPred'] = test_scores_df['HomeTheilPred']-test_scores_df['RoadTheilPred']

In [20]:
from sklearn.metrics import mean_squared_error

In [24]:
r2_score(test_scores_df['HomeTrue'], test_scores_df['HomeBayPred'])

-0.008570206101269573

In [21]:
from sklearn.metrics import r2_score

In [112]:
test_scores_df.head()

,HomeTrue,RoadTrue,HomeBayPred,RoadBayPred,HomeArdPred,RoadArdPred,HomeHuberPred,RoadHuberPred,HomeSgdPred,RoadSgdPred,HomeTheilPred,RoadTheilPred,DeltaTrue,DeltaBayPred,DeltaArdPred,DeltaHuberPred,DeltaSgdPred,DeltaTheilPred
0,121,111,115.205701,109.599037,115.413537,109.246479,120.891681,109.205619,114.557517,109.574995,114.910199,109.235234,10,5.606664,6.167058,11.686061,4.982521,5.674965
1,134,107,111.454809,112.554106,111.339088,112.533289,104.423967,111.874268,110.508577,112.416991,110.978009,112.107814,27,-1.099297,-1.194201,-7.450301,-1.908414,-1.129805
2,120,118,113.318449,109.096017,112.337554,110.084261,121.438268,108.208256,111.635351,107.866980,112.509482,108.507322,2,4.222431,2.253293,13.230012,3.768371,4.002160
3,101,112,104.401385,107.951967,104.391181,108.123483,105.686122,107.468946,104.060001,108.065829,104.051463,107.557211,-11,-3.550582,-3.732302,-1.782824,-4.005828,-3.505748
4,122,110,114.464933,117.115551,114.574398,116.957803,105.844943,116.583552,114.772258,116.973410,114.210530,116.675360,12,-2.650618,-2.383405,-10.738609,-2.201152,-2.464830


In [22]:
from sklearn.metrics import mean_absolute_error

In [114]:
mean_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaSgdPred'])

11.5575714691747

In [123]:
mae_models = {}
mae_models['BayRidgeRegress'] = mean_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaBayPred'])
mae_models['ArdRegress'] = mean_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaArdPred'])
mae_models['HuberRegress'] = mean_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaHuberPred'])
mae_models['SgdRegress'] = mean_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaSgdPred'])
mae_models['TheilSenRegress'] = mean_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaTheilPred'])

In [116]:
mae_models

{'bay': 11.412105899737698,
 'ard': 11.53162103547274,
 'huber': 11.980774434477507,
 'sgd': 11.5575714691747,
 'theil': 11.447859944462495}

In [124]:
r2_models = {}
r2_models['BayRidgeRegress'] = r2_score(test_scores_df['DeltaTrue'], test_scores_df['DeltaBayPred'])
r2_models['ArdRegress'] = r2_score(test_scores_df['DeltaTrue'], test_scores_df['DeltaArdPred'])
r2_models['HuberRegress'] = r2_score(test_scores_df['DeltaTrue'], test_scores_df['DeltaHuberPred'])
r2_models['SgdRegress'] = r2_score(test_scores_df['DeltaTrue'], test_scores_df['DeltaSgdPred'])
r2_models['TheilSenRegress'] = r2_score(test_scores_df['DeltaTrue'], test_scores_df['DeltaTheilPred'])
r2_models

{'BayRidgeRegress': 0.012020628262219835,
 'ArdRegress': -0.008011341398253435,
 'HuberRegress': -0.06740326238674599,
 'SgdRegress': -0.009359596502744205,
 'TheilSenRegress': 0.006534025499589813}

In [125]:
mse_models = {}
mse_models['BayRidgeRegress'] = mean_squared_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaBayPred'])
mse_models['ArdRegress'] = mean_squared_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaArdPred'])
mse_models['HuberRegress'] = mean_squared_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaHuberPred'])
mse_models['SgdRegress'] = mean_squared_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaSgdPred'])
mse_models['TheilSenRegress'] = mean_squared_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaTheilPred'])
mse_models

{'BayRidgeRegress': 209.76755076461762,
 'ArdRegress': 214.02073391081785,
 'HuberRegress': 226.6308127822504,
 'SgdRegress': 214.30699512149218,
 'TheilSenRegress': 210.9324649890016}

In [23]:
from sklearn.metrics import median_absolute_error

In [126]:
medae_model = {}
medae_model['BayRidgeRegress'] = median_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaBayPred'])
medae_model['ArdRegress'] = median_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaArdPred'])
medae_model['HuberRegress'] = median_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaHuberPred'])
medae_model['SgdRegress'] = median_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaSgdPred'])
medae_model['TheilSenRegress'] = median_absolute_error(test_scores_df['DeltaTrue'], test_scores_df['DeltaTheilPred'])
medae_model

{'BayRidgeRegress': 9.140330776526199,
 'ArdRegress': 9.256028585710403,
 'HuberRegress': 10.005108106275635,
 'SgdRegress': 9.413887799478275,
 'TheilSenRegress': 9.196041293502674}

In [127]:
regress_metrics = {}
regress_metrics["MAE"] = mae_models
regress_metrics["R2"] = r2_models
regress_metrics["MSE"] = mse_models
regress_metrics["MedAE"] = medae_model

In [128]:
db.model_regress_data.insert_one(regress_metrics)